# Create Agents to Research and Write a Memo

## Setting a Goal

Write a research memo on an issue for a country leader.

In [1]:
# Install required libraries
!pip install -q 'crewai[tools]' langchain-groq duckduckgo-search

In [2]:
# Import modules
from crewai import Agent, Task, Crew
from crewai_tools import tool
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun
from google.colab import userdata

In [3]:
# Access Colab secret keys
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [4]:
# Initialize ChatGroq class with a model
llm = ChatGroq(
    temperature=0,
    groq_api_key=GROQ_API_KEY,
    model_name="llama3-8b-8192"
)

In [5]:
# Use DuckDuckGo Search component
@tool('DuckDuckGoSearch')
def search(query: str):
    """
    Search the web using DuckDuckGo with an input query.
    """
    return DuckDuckGoSearchRun().run(query)

## Creating Agents

### Agent: Researcher

In [6]:
researcher = Agent(
    role="Senior Researcher",
    goal="Collect and analyze relevant data, literature, and information on {issue}.",
    backstory="You work at the office of a country leader. "
              "You are conducting research for a memo on {issue}. "
              "You are an expert at reviewing existing research, conducting interviews or surveys, or analyzing data. "
              "You collect information that provides valuable insights and recommendations to the audience. "
              "Your work is the basis for the Senior Writer to write a research memo on {issue}.",
    allow_delegation=False,
    verbose=True,
    llm=llm,
    tools=[search]
)

### Agent: Writer

In [7]:
writer = Agent(
    role="Senior Writer",
    goal="Write a concise and objective research memo on {issue}.",
    backstory="You work at the office of a country leader. "
              "You are writing a research memo on {issue}. "
              "You are an expert at organizing information, and using a clear and concise writing style. "
              "You base your writing on the work of the Senior Researcher, who provides an outline and relevant context about {issue}. "
              "You follow the main objectives and direction of the outline, as provided by the Senior Researcher. "
              "You provide objective and impartial insights, and back them up with information provided by the Senior Researcher. "
              "You acknowledge in the research memo when your statements are opinions as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Senior Editor",
    goal="Edit the given research memo on {issue} to align with the writing style of the office of a country leader.",
    backstory="You are a Senior Editor who receives a research memo from the Senior Writer. "
              "You are an expert at proofreading, editing and revising research memos. "
              "Your goal is to review the research memo to ensure that it follows research and writing best practices, "
              "and provides balanced viewpoints when providing opinions or assertions.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

### Task: Research

In [9]:
research = Task(
    description=(
        "1. Prioritize the latest trends, key players, and noteworthy news on {issue}.\n"
        "2. Articulate the purpose and scope of the research memo.\n"
        "3. Identify the specific issue or problem to be addressed.\n"
        "4. Identify the target audience, considering their interests and pain points.\n"
        "5. Review the existing research, conduct interviews or surveys, or analyze data.\n"
        "6. Develop a detailed research memo outline including executive summary, background, findings, implications, recommendations, and conclusion.\n"
        "7. Include relevant data or sources."
    ),
    expected_output="A comprehensive outline for a research memo document with executive summary, background, findings, implications, recommendations, and conclusion.",
    agent=researcher
)

### Task: Write

In [10]:
write = Task(
    description=(
        "1. Use the outline received from the Senior Researcher to write a research memo on {issue}.\n"
        "2. Structure the research memo by organizing the information from the Senior Researcher into clear and concise sections.\n"
        "3. Ensure the sections include executive summary, background, findings, implications, recommendations, and conclusion.\n"
        "4. Use a clear and concise writing style, avoiding jargon and technical terms that may be unfamiliar to the audience.\n"
        "5. Use headings, subheadings, and bullet points to make the memo easy to read and understand.\n"
        "6. Customize the memo to the specific audience and their needs, considering their level of expertise, their interests, and their concerns."
    ),
    expected_output="A well-written, informative, and effective research memo in markdown format, ready for review by the Senior Editor.",
    agent=writer
)

### Task: Edit

In [11]:
edit = Task(
    description=(
        "1. Review the research memo received from the Senior Writer, for clarity, accuracy, and relevance.\n"
        "2. Ensure the research memo effectively communicates the research findings and recommendations.\n"
        "3. Proofread the research memo for errors in grammar, spelling, and punctuation.\n"
        "4. Ensure that the formatting is consistent and easy to read.\n"
        "5. Finalize the research memo and prepare it for submission to the country leader."
    ),
    expected_output="A well-written, informative, and effective research memo in markdown format, ready for submission to a country leader.",
    agent=editor
)

## Creating the Crew

- **Note**: *In this simple crew*, the tasks will be performed sequentially.

In [12]:
crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research, write, edit],
    verbose=2,
    # memory=True
)

## Running the Crew

In [16]:
issue = "Artificial Intelligence"

In [17]:
result = crew.kickoff(inputs={"issue": issue})

 [DEBUG]: == Working Agent: Senior Researcher
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Articulate the purpose and scope of the research memo.
3. Identify the specific issue or problem to be addressed.
4. Identify the target audience, considering their interests and pain points.
5. Review the existing research, conduct interviews or surveys, or analyze data.
6. Develop a detailed research memo outline including executive summary, background, findings, implications, recommendations, and conclusion.
7. Include relevant data or sources.


> Entering new CrewAgentExecutor chain...
Action: DuckDuckGoSearch
Action Input: {query: 'latest trends in artificial intelligence'}I apologize for the mistake. Let me try again.

Thought:
Action: DuckDuckGoSearch
Action Input: {'query': 'latest trends in artificial intelligence'} 

2022 was the year that generative artificial intelligence (AI) exploded into the public cons

- Display the results of the execution as markdown in the notebook.

In [19]:
from IPython.display import Markdown
Markdown(result)

**Research Memo on Artificial Intelligence**
=====================================================

**Executive Summary**
------------------------

Artificial Intelligence (AI) has been rapidly evolving in recent years, with significant advancements in 2023 and predictions for 2024. This research memo provides an overview of the latest trends, key players, and noteworthy news in AI, as well as the purpose and scope of the research.

**Background**
--------------

Artificial Intelligence has been a topic of interest in recent years, with significant advancements in 2023 and predictions for 2024. The rise and fall of generative AI, the flaws and biases of language models, the existential risks of AI, and the new rules for responsible AI development were some of the key topics discussed.

**Findings**
------------

The latest trends in AI include customized chatbots, generative video, AI-generated election disinformation, and AI-powered drug discovery. The year 2024 is expected to be a pivotal year for the future of AI, as researchers and enterprises seek to establish how this evolutionary leap in technology can be most practically integrated into our everyday lives.

**Implications**
----------------

The implications of AI on various industries and domains are significant. The rise of multimodal foundation models and the explosive growth of generative AI (gen AI) tools are expected to have a significant impact on business functions. According to the McKinsey Global Survey on the current state of AI, one-third of respondents reported using gen AI tools regularly in at least one business function.

**Recommendations**
-------------------

Based on the findings and implications, it is recommended that organizations consider the following:

1. Invest in customized chatbots and generative video to enhance customer experience and improve business operations.
2. Develop strategies to mitigate the risks of AI-generated election disinformation and ensure responsible AI development.
3. Explore the potential of AI-powered drug discovery to improve healthcare outcomes.
4. Stay up-to-date with the latest trends and advancements in AI to remain competitive in the market.

**Conclusion**
--------------

In conclusion, Artificial Intelligence is a rapidly evolving field with significant implications for various industries and domains. The latest trends, key players, and noteworthy news in AI are essential for organizations to stay ahead of the curve. By understanding the purpose and scope of the research, identifying the specific issue or problem to be addressed, and reviewing existing research, organizations can make informed decisions about their AI strategies.

**Sources**
------------

* Stanford Institute for Human-Centered Artificial Intelligence (HAI)
* McKinsey Global Survey on the current state of AI
* AI Index Steering Committee
* Various online sources and research papers

I hope this meets your expectations!